<a href="https://colab.research.google.com/github/Santiago-Quinteros/Applied-Mathematics-Internship/blob/main/VTK_Conyle_Sphere_and_Ellipsoid_modelization_Anonimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
delete = 'mesh_folder'
shutil.rmtree(delete)

!unzip mesh_folder.zip -d mesh_folder #For Collab

In [ ]:
#install libraries
!pip install vtk
!pip install jax

In [ ]:
#import libraries
import os
import numpy as np
import vtk
from scipy import optimize
from scipy.optimize import fmin_bfgs
from jax import grad

#check if I dont need some of the deleted ones from: https://github.com/cjekel/cjekel.github.io/blob/master/assets/2021-12-04/non-linear-ellipsoid-fit.py

In [ ]:
#Read meshes
def LoadMesh(filename):
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(filename)
    reader.ReadAllScalarsOn()
    reader.ReadAllVectorsOn()
    reader.Update()
    mesh = reader.GetOutput()
    return mesh

In [ ]:
#Fit meshes to a sphere
def FitSphere(mesh):
    points = np.array([mesh.GetPoint(i) for i in range(mesh.GetNumberOfPoints())])
    b = np.zeros(len(points))
    A = np.zeros((len(points), 4))

    for i in range(len(points)):
        b[i] = points[i][0]**2 + points[i][1]**2 + points[i][2]**2
        A[i] = [2 * points[i][0], 2 * points[i][1], 2 * points[i][2], 1]

    res = np.linalg.lstsq(A, b, rcond=None)[0]
    center = vtk.vtkVector3d(res[0], res[1], res[2])
    radius = np.sqrt(res[3] + res[0]**2 + res[1]**2 + res[2]**2)

    return center, radius

In [ ]:
#Fit meshes to an Elipsoid
def FitElipsoid(mesh):
  import numpy as np
  import matplotlib.pyplot as plt
  from mpl_toolkits.mplot3d import Axes3D
  import jax.numpy as jnp
  from jax import grad
  from jax import random
  from jax.config import config
  config.update('jax_enable_x64', True)
  key = random.PRNGKey(0)

  points = np.array([mesh.GetPoint(i) for i in range(mesh.GetNumberOfPoints())])
  x=points[:,0]
  y=points[:,1]
  z=points[:,2]

  center,radius=FitSphere(mesh)
  gamma_guess = np.random.random(6)
  gamma_guess[0] = center[0] #x0
  gamma_guess[1] = center[1] #y0
  gamma_guess[2] = center[2] #z0
  gamma_guess[3] = radius #a
  gamma_guess[4] = radius #b
  gamma_guess[5] = radius #c
  gamma_guess = jnp.array(gamma_guess)

  #M-Tukey  ----------------------------------------------- Currently ommited due to computational cost and poor results
  def TukeyMEstimator(da):
    sigma=6
    ei=da
    emed=jnp.median(da)
    abseiemed= jnp.abs(ei-emed)
    MAD=jnp.median(abseiemed)
    condition=sigma*MAD
    wi= []
    for i in range(len(ei)):
      if abseiemed[i]<=condition:
        wi.append((1-(abseiemed[i]/condition)**2)**2)
      else:
        wi.append(0)
    return jnp.array(wi)

  def predict(gamma):
      # compute f hat
      x0 = gamma[0]
      y0 = gamma[1]
      z0 = gamma[2]
      a2 = gamma[3]**2
      b2 = gamma[4]**2
      c2 = gamma[5]**2
      zeta0 = (x - x0)**2 / a2
      zeta1 = (y - y0)**2 / b2
      zeta2 = (z - z0)**2 / c2
      return zeta0 + zeta1 + zeta2 -1

  def loss(g):
      # compute mean squared error
      da = predict(g)                      #f(a,pi)
      da2= jnp.square(da)                  #f(a,pi)^2
      #wi=TukeyMEstimator(da)               #wi
      #wda=jnp.multiply(wi,da2)             #wi.f(a,pi)^2
      #mse = wda.mean()                     #(1/n)*S(wi.f(a,pi)^2)
      mse =  da2.mean()
      return mse
      #Estimatror M-Tukey helped eliminating the unusual values (which wont be present as we are using an SSM) and reducing the bias caused by the use of the algebraic distance
      #With the tests I performed, using this resulted in 8 minutes for every new ellipsoid (which before was done in less than a second) and in results that did't vary much from the initial estimation

  from scipy.optimize import fmin_bfgs

  res = fmin_bfgs(
      loss,
      gamma_guess,
      fprime=grad(loss),
      norm=2.0,
      gtol=1e-4,
      maxiter=100,
      disp=False
  )

  center= vtk.vtkVector3d(res[0],res[1],res[2])
  radius= vtk.vtkVector3d(res[3],res[4],res[5])
  return (center,radius)

#Mesure the mse between a sphere and the mesh
  def loss(g):
      # compute mean squared error
      pred = predict(g)
      target = jnp.ones_like(pred)
      mse = jnp.square(pred-target).mean()
      return mse

#Mesure the mse between an ellipsoid and the mesh
  def mse
  def loss(g):
      # compute mean squared error
      pred = predict(g)
      target = jnp.ones_like(pred)
      mse = jnp.square(pred-target).mean()
      return mse

        def predict(gamma):
      # compute f hat
      x0 = gamma[0]
      y0 = gamma[1]
      z0 = gamma[2]
      a2 = gamma[3]**2
      b2 = gamma[4]**2
      c2 = gamma[5]**2
      zeta0 = (x - x0)**2 / a2
      zeta1 = (y - y0)**2 / b2
      zeta2 = (z - z0)**2 / c2
      return zeta0 + zeta1 + zeta2
      

In [ ]:
#Generate a .vtk Sphere and Ellipsoid adapted to the data for all patients with the center.
folder_path = "mesh_folder"
patients = [f for f in os.listdir(folder_path) if (os.path.isdir(os.path.join(folder_path, f)) and len(f) == 3 and f.isnumeric())]
patients.sort(key=lambda x: int(x))
space='inModelSpace'
condyle=["Medial","Lateral"]
condylefile=['condile_file_medial.vtk','condile_file_lateral.vtk']



def CreateSphere(S_center,S_radius,condyle):

  sphere = vtk.vtkSphereSource()
  sphere.SetCenter(S_center)
  sphere.SetRadius(S_radius)

  sphere.SetThetaResolution(100)
  sphere.SetPhiResolution(100)

  writer = vtk.vtkPolyDataWriter()
  writer.SetFileName(os.path.join(route,condyle+"Sphere"+".vtk"))
  writer.SetInputConnection(sphere.GetOutputPort())
  writer.Write()

  #Center for visualization:
  sphereC = vtk.vtkSphereSource()
  sphereC.SetCenter(S_center)
  sphereC.SetRadius(1)

  sphereC.SetThetaResolution(100)
  sphereC.SetPhiResolution(100)

  writer = vtk.vtkPolyDataWriter()
  writer.SetFileName(os.path.join(route,condyle+"SphereCenter"+".vtk"))
  writer.SetInputConnection(sphereC.GetOutputPort())
  writer.Write()

  return

def CreateEllipsoid(E_center,E_radius,condyle):
  ellipsoid = vtk.vtkSphereSource()
  ellipsoid.SetCenter(0,0,0)
  ellipsoid.SetRadius(1.0)
  ellipsoid.SetPhiResolution(100)
  ellipsoid.SetThetaResolution(100)

  transform = vtk.vtkTransform()
  transform.Translate(E_center)
  transform.Scale(E_radius)

  transform_filter = vtk.vtkTransformPolyDataFilter()
  transform_filter.SetInputConnection(ellipsoid.GetOutputPort())
  transform_filter.SetTransform(transform)

  writer = vtk.vtkPolyDataWriter()
  writer.SetFileName(os.path.join(route,condyle+"Ellipsoid"+".vtk"))
  writer.SetInputConnection(transform_filter.GetOutputPort())
  writer.Write()

  #Center for visualization:
  ellipsoidC = vtk.vtkSphereSource()
  ellipsoidC.SetCenter(E_center)
  ellipsoidC.SetRadius(1)

  ellipsoidC.SetThetaResolution(100)
  ellipsoidC.SetPhiResolution(100)

  writer = vtk.vtkPolyDataWriter()
  writer.SetFileName(os.path.join(route,condyle+"EllipsoidCenter"+".vtk"))
  writer.SetInputConnection(ellipsoidC.GetOutputPort())
  writer.Write()
  return

for p in range(len(patients)):
    #make these the leg folders (osea left y/o right)
    legs= [f for f in os.listdir(os.path.join(mesh_folderpatients[p])) if os.path.isdir(os.path.join(os.path.join(mesh_folder,patients[p]), f))]
    for l in range(len(legs)):
        for c in range(len(condylefile)):
            route = os.path.join(mesh_folder,patients[p],legs[l],'localization',space) #Dossier avec le maiallage
            file = condylefile[c]
            filename = os.path.join(route, file)
            mesh = LoadMesh(filename)
            S_center, S_radius = FitSphere(mesh)
            E_center, E_radius = FitElipsoid(mesh)
            CreateSphere(S_center,S_radius,condyle[c])
            CreateEllipsoid(E_center,E_radius,condyle[c])
            print('patient:',patients[p],'|','Leg:',legs[l],'|', 'Condyle:', 'Medial' if c==0 else 'Lateral','\n','\n',
                  'Sphere:','\n',
                  'Center:',S_center,'\n','Radius:',S_radius,'\n','\n',
                  'Ellipsoid:','\n',
                  'Center:',E_center,'\n','Radius:',E_radius,'\n',
                  '________________________________________________________________________________')

patient: 002 | Leg: Left | Condyle: Medial 
 
 Sphere: 
 Center: [53.43544245673525, 2.0476559418500955, -194.30794325788037] 
 Radius: 24.285250236161012 
 
 Ellipsoid: 
 Center: [53.18708162008069, -0.2581203265737093, -194.43538388186224] 
 Radius: [28.87755769352634, 26.573446504617912, 24.35664745244991] 
 ________________________________________________________________________________
patient: 002 | Leg: Left | Condyle: Lateral 
 
 Sphere: 
 Center: [-6.000623918376706, 3.6524632347780397, -195.84332974166594] 
 Radius: 24.314873093857408 
 
 Ellipsoid: 
 Center: [-6.0006320115246705, 3.6524205036616944, -195.84332315500313] 
 Radius: [24.316050434540173, 24.322600975862425, 24.316039941747224] 
 ________________________________________________________________________________
patient: 002 | Leg: Right | Condyle: Medial 
 
 Sphere: 
 Center: [52.71659296890802, 3.308025923710005, -195.24619498678913] 
 Radius: 22.128857325345106 
 
 Ellipsoid: 
 Center: [52.527466789167, -0.895813

In [ ]:
#DownloadGeneratedData
import zipfile

zip_file = zipfile.ZipFile('Generated.zip', 'w', zipfile.ZIP_DEFLATED)
for root, dirs, files in os.walk('mesh_folder'):
    for file in files:
        zip_file.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join("mesh_folder", '..')))
zip_file.close()